# Twitter Sentiment Analyzer

In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


## Importing Modules

In [3]:
import customtkinter as ctk
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

## Loading pre-trained Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment') 

## GUI

In [5]:
class SentimentAnalyzer(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        self.title(" Sentiment Analyzer")
        self.geometry("600x400")
        
        # Create the title label
        self.title_label = ctk.CTkLabel(self, text="Sentiment Analyzer", font=("Arial", 28, "bold"))
        self.title_label.pack(pady=20)
        # Create labels, entry box, and buttons
        self.text_label = ctk.CTkLabel(self, text="Paste your Tweet/Review below:", font=("Arial", 16))
        self.text_label.pack(pady=10)

        self.text_entry = ctk.CTkTextbox(self, width=600, height=50, font=("Arial", 14))
        self.text_entry.pack(pady=10)

        # Create a frame to hold the buttons
        button_frame = ctk.CTkFrame(self)
        button_frame.pack(pady=10)

        self.analyze_button = ctk.CTkButton(button_frame, text="Analyze", command=self.analyze_sentiment)
        self.analyze_button.pack(side="left", padx=10)

        self.clear_button = ctk.CTkButton(button_frame, text="Clear", command=self.clear_text)
        self.clear_button.pack(side="right", padx=10)

        # Frame for sentiment score and probabilities
        self.output_frame = ctk.CTkFrame(self)
        self.output_frame.pack(pady=30)

        self.sentiment_label = ctk.CTkLabel(self.output_frame, text="Sentiment Analysis", font=("Arial", 18,'bold'))
        self.sentiment_label.pack(side="left", padx=10, anchor='w')

        self.probabilities_label = ctk.CTkLabel(self.output_frame, text="Sentiment Probabilities", font=("Arial", 18,'bold'))
        self.probabilities_label.pack(side="right", padx=10, anchor='e')
        
    def analyze_sentiment(self):
        text = self.text_entry.get("1.0", "end-1c")
        if text:
            # Tokenize and get model predictions
            tokens = tokenizer.encode(text, return_tensors='pt')
            with torch.no_grad():
                result = model(tokens)
            
            # Extract logits and calculate probabilities
            logits = result.logits
            probabilities = torch.softmax(logits, dim=-1).squeeze().tolist()
            sentiment_score = int(torch.argmax(logits)) + 1

            # Map score to sentiment
            sentiment_map = {
                1: "Very Negative",
                2: "Negative",
                3: "Neutral",
                4: "Positive",
                5: "Very Positive"
            }

            sentiment_label = sentiment_map.get(sentiment_score, "Unknown")

            # Format and display output
            sentiment_text = f"Sentiment Analysis\n\n Sentiment Score: {sentiment_score}\nSentiment: {sentiment_label}"
            
            # Map probabilities to sentiment labels
            sentiment_labels = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
            label_probability_pairs = list(zip(sentiment_labels, probabilities))
            sorted_pairs = sorted(label_probability_pairs, key=lambda x: x[1], reverse=True)
            probabilities_text = "\n".join(f"{label:<20}: {prob:.2f}" for label, prob in sorted_pairs)

            self.sentiment_label.configure(text=sentiment_text)
            self.probabilities_label.configure(text=f"Probabilities: \n\n{probabilities_text}")
        else:
            self.sentiment_label.configure(text="Please enter text for analysis.")
            self.probabilities_label.configure(text="")

    def clear_text(self):
        self.text_entry.delete("1.0", "end")
        self.sentiment_label.configure(text="")
        self.probabilities_label.configure(text="")

In [ ]:
if __name__ == "__main__":
    app = SentimentAnalyzer()
    app.mainloop()